# Fetching codes

In [ ]:
!git clone https://github.com/minhnguyent546/nmt-en-vi.git

In [ ]:
%cd nmt-en-vi

In [3]:
!pip install -q datasets underthesea contractions

# Configuration

In [4]:
from nmt.utils.config import get_config
from pathlib import Path

storage_dir = '/kaggle/working/storage'
Path(storage_dir).mkdir(parents=True, exist_ok=True)

config = get_config('./config/config.yaml')

In [5]:
!mkdir -p /kaggle/working/datasets/iwslt2015-en-vi
!cp /kaggle/input/iwslt2015-en-vi/*.json /kaggle/working/datasets/iwslt2015-en-vi

In [5]:
config['checkpoints_dir'] = f'{storage_dir}/checkpoints'
config['experiment_name'] = f'{storage_dir}/runs/nmt'
config['dataset_save_path'] = f'{storage_dir}/datasets/iwslt2015-en-vi'
config['num_epochs'] = 10

dataset_dir = '/kaggle/working/datasets/iwslt2015-en-vi'
config['data_files'] = {
    'train': f'{dataset_dir}/train.json',
    'test': f'{dataset_dir}/test.json',
    'validation': f'{dataset_dir}/validation.json',
}

In [ ]:
config

# Preprocessing

In [ ]:
from nmt.preprocess import preprocess
preprocess(config)

# Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
! ! command -v ngrok &>/dev/null && \
        wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz && \
        tar -xvzf ngrok-v3-stable-linux-amd64.tgz -C /usr/local/bin

!ngrok authtoken '<YOUR_NGROK_AUTH_TOKEN>'

import os
import multiprocessing
pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd,))
                        for cmd in [
                            f'tensorboard --logdir {config["experiment_name"]} --host 0.0.0.0 --port 6007 &',
                            "/usr/local/bin/ngrok http 6007 &"
                        ]]

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard is running at:', json.load(sys.stdin)['tunnels'][0]['public_url'])"

# Training

In [ ]:
from nmt.train import train_model
train_model(config)

# Testing

In [ ]:
config['compute_bleu_kwargs']['max_steps'] = None
config['compute_bleu_kwargs']['beam_size'] = 4
config['compute_bleu_kwargs']

In [ ]:
from nmt.test import test_model
test_model(config)

In [ ]:
!pkill jupyter